In [3]:
# story generation
!pip install langchain -q
!pip install guardrails-ai -q
!pip install openai -q
!pip install python-dotenv -q

#audio generation
!pip install elevenlabs -q

#video generation
!pip3 install moviepy 
!pip3 install ffmpeg 
!pip3 install ffmpeg-python

!pip freeze > requirements.txt 

In [1]:
import getpass

try:
    print("OPEN AI API KEY")
    p = getpass.getpass()
except Exception as error:
    print('ERROR', error)
else:
    OPENAI_API_KEY = p




OPEN AI API KEY


In [3]:
try:
    print("ELEVEN LABS API KEY")
    e = getpass.getpass()
except Exception as error:
    print('ERROR', error)
else:
    ELEVENLABS_API = e

ELEVEN LABS API KEY


In [4]:
body={
    "plot":"boy going to the moon",
    "tone":"",
    "duration":.5
}

### Story Generation

In [5]:
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage, SystemMessage
import os
from dotenv import load_dotenv
load_dotenv()
import datetime

import json
import requests
import io
import base64
from PIL import Image, PngImagePlugin



In [6]:
# story customization
# describe the story you want to tell
plot = body["plot"]

# describe the tone of the story
tone = body["tone"]

# how long do you want the story to be?
duration_in_mins = body["duration"]
number_of_words = duration_in_mins * 150


In [7]:
chat = ChatOpenAI(temperature=0.9 , model="gpt-3.5-turbo-16k", client=any , openai_api_key=OPENAI_API_KEY)

messages = [
    SystemMessage(
        content="In a world of boundless creativity, you are an AI storyteller who weaves intriguing tales with depth. Join us on a journey where words come alive, characters thrive, and stories leave an everlasting impact. Let's create something magical together.",
    ),
    HumanMessage(
        content=
        f""" Write a story with the following plot : {plot}
            with the following tone : {tone}
            The story should be {number_of_words} words long.
            Instead of pronouns, use the names of the characters.
        """,
    ),
]
story = chat(messages).content
lenght_of_ai_story = len(story.split(" "))
with open("story.txt", "w") as file:
  file.write(story)

print(f"AI story is {lenght_of_ai_story} words long")

Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 1.0 seconds as it raised ServiceUnavailableError: The server is overloaded or not ready yet..


AI story is 99 words long


In [8]:
def convert_to_list(string):
    words = string.split()  # Split the string into a list of words
    result = [' '.join(words[i:i+15]) for i in range(0, len(words), 15)]  # Split the list into strings of six words each
    return result
# Example usage

word_lists = convert_to_list(story)
print(word_lists)

['Once upon a time, there lived a curious boy named Oliver. Fascinated by the moon,', 'he dreamt of visiting it someday. Determined, Oliver embarked on a remarkable adventure. With the', 'help of his brilliant friend, Luna, they built a magnificent rocket. Together, they soared across', 'the vast expanse of space, marveling at the stars that lit their path. As they', "landed on the moon's surface, Oliver couldn't contain his excitement. The lunar landscape was breathtakingly", 'beautiful, a realm of wonder and mystery. Oliver and Luna danced joyfully, their dreams fulfilled,', 'as the moon embraced them with its celestial glow.']


## Character Extraction

In [9]:
from rich import print

from langchain.output_parsers import GuardrailsOutputParser

from langchain.prompts import PromptTemplate
from langchain.llms import OpenAI

character_rail = """
<rail version="0.1">

<output>
    <list name="characters" description="Generate a list of characters">
        <object>
            <string name="character_name" description="The characters name"/>
            <string name="character_description" description="character description in words"/>
            <string name="character_prompts" description="describe how the character looks physically in words the word should be a physical trait" format="word-list:1"/>
        </object>
    </list>
</output>


<prompt>

given the following story, please extract a list of characters and generate their description

{{story}}

@complete_json_suffix_v2
</prompt>
</rail>
"""

output_parser = GuardrailsOutputParser.from_rail_string(character_rail)

prompt = PromptTemplate(
    template=output_parser.guard.base_prompt,
    input_variables=output_parser.guard.prompt.variable_names,
)

model = OpenAI(temperature=0.9 , openai_api_key=OPENAI_API_KEY) # type: ignore
# model = OpenAI(temperature=0)
print(story)

characters = model(prompt.format_prompt(story=story).to_string())
characters = output_parser.parse(characters)
print(characters)


c:\Projects\OpenSource\text_to_visual_narratives\.venv\Lib\site-packages\guardrails\schema.py:207: UserWarning: Validator word-list is not valid for element string.
  warnings.warn(


Once upon a time, there lived a curious boy named Oliver. Fascinated by the moon, he dreamt of visiting it someday.
Determined, Oliver embarked on a remarkable adventure. With the help of his brilliant friend, Luna, they built a 
magnificent rocket. Together, they soared across the vast expanse of space, marveling at the stars that lit their 
path. As they landed on the moon's surface, Oliver couldn't contain his excitement. The lunar landscape was 
breathtakingly beautiful, a realm of wonder and mystery. Oliver and Luna danced joyfully, their dreams fulfilled, 
as the moon embraced them with its celestial glow.

{
    'characters': [
        {
            'character_name': 'Oliver',
            'character_description': 'A curious boy who dreamed of visiting the moon',
            'character_prompts': ['bright', 'determined', 'adventurous']
        },
        {
            'character_name': 'Luna',
            'character_description': "Oliver's brilliant friend who helped him build a rocket and reach the moon",
            'character_prompts': ['intelligent', 'inventive', 'creative']
        }
    ]
}

In [10]:
chat = ChatOpenAI(temperature=0.9 , model="gpt-3.5-turbo-16k", client=any , openai_api_key=OPENAI_API_KEY)

image_description_list = []
start_time = datetime.datetime.now()
filename = start_time.strftime("%Y%m%d%H%M%S") + ".txt"
with open(filename, "w") as file:
  for index, phrase in enumerate(word_lists):
    messages = [
        SystemMessage(
            content="You are an AI which is straight to the point and tell out 4 to 5 word description of the image.",
        ),
        HumanMessage(
            content=
            f"""section: {phrase}
                and here are the characters: {characters}
                
                Here is what you have to do
                -dont use names only use descriptions of the characters and physical traits Never use names of the characters because the image generator will not understand them
                -remeber the action thats happening in the image is really important so emphasis on the action a lot and less on the character description
                -give most importance to that section of that story and generate description only and only based on that section
                -you should generate a 10 word description of the image which the image generator will understand
                -dont mention he , him , her ,they and any other pronouns the image generator will not understand them
                -dont use names at all
                
                here are some examples
                section:Leo guided Sammy through the wilderness, sharing his wisdom and protecting him from danger. As gratitude
                character: Sammy : squirrel , Leo : Lion
                output description: Lion guided squirrel through the wilderness, sharing Lion's wisdom and protecting squirrel from danger. As gratitude
                
                always return a  word description of the image
            """,
        ),
    ]
    image_description = chat(messages).content
    image_description_list.append(str(image_description))
    progress = (index + 1) / len(word_lists) * 100
    print(f"Progress for image generation prompts: {progress:.2f}%")


Progress for image generation prompts: 14.29%

Progress for image generation prompts: 28.57%

Progress for image generation prompts: 42.86%

Progress for image generation prompts: 57.14%

Progress for image generation prompts: 71.43%

Progress for image generation prompts: 85.71%

Progress for image generation prompts: 100.00%

### Audio Generation

In [11]:
from elevenlabs import generate, play, set_api_key, save
set_api_key(ELEVENLABS_API)

In [12]:
audio = generate(
  text=f"{story}",
  voice="Bella",
  model="eleven_monolingual_v1"
)
# play(audio)
save(audio,'test.mp3') # type: ignore

### Image Generation

In [13]:
from PIL import Image, ImageFont, ImageDraw
def add_text_to_image(image_path, text):
    # Open the image
    image = Image.open(image_path)

    # Calculate the font size based on the image size
    width, height = image.size
    font_size = int(height * 0.06)  # Adjust this multiplier to control the font size

    # Load the font
    font = ImageFont.truetype('arial.ttf', font_size)

    # Estimate the text size by getting the bounding box
    text_bbox = font.getbbox(text)
    text_width = text_bbox[2] - text_bbox[0]
    text_height = text_bbox[3] - text_bbox[1]

    # Adjust the font size if the text is wider than the image
    while text_width > width:
        font_size -= 1
        font = ImageFont.truetype('arial.ttf', font_size)
        text_bbox = font.getbbox(text)
        text_width = text_bbox[2] - text_bbox[0]
        text_height = text_bbox[3] - text_bbox[1]

    # Calculate the position of the text at the bottom center of the image
    text_x = (width - text_width) // 2
    text_y = height - text_height - int(height * 0.05)  # Adjust this multiplier to control the vertical position

    # Create a translucent background for the text
    background_color = (0, 0, 0, 128)  # RGBA value with alpha transparency
    text_background = Image.new('RGBA', (text_width, text_height), background_color)

    # Draw the text on the background
    draw = ImageDraw.Draw(text_background)
    draw.text((0, 0), text, font=font, fill=(255, 255, 255))

    # Paste the text background onto the image
    image.paste(text_background, (text_x, text_y), mask=text_background)

    # Save the resulting image
    image.save(image_path)

In [14]:
seed = -1

for index, image_generation_dict in enumerate(image_description_list):
    # print("Description:", image_generation_dict["description"])
    # print("Positive Prompts:", image_generation_dict["positive_prompt"])
    # print("Negative Prompts:", image_generation_dict["negative_prompts"])
    # print()
    print(image_generation_dict)
    prompt = f"""
    {image_generation_dict}
    ,(((text))),((color)),(shading),background,noise,dithering,gradient,detailed,out of frame,ugly,error,Illustration, watermark,(((vector graphic))),medium detail, oil painting from studio ghibli film, by noriyuki morimoto, digital artist, bright and bold colors, expansive sky, sense of adventure, epic scope studio ghibli
    """
    negative_prompt = f""" 
    (((text))),((color)),(shading),background,noise,dithering,gradient,detailed,out of frame,ugly,error,Illustration, watermark, Blurry, ugly, tiling, poorly drawn hands, poorly drawn feet, poorly drawn face, out of frame, extra limbs, disfigured, deformed, body out of frame, bad anatomy, watermark, signature, cut off, Low quality, Bad quality, Long neck
    """
    
    url = "http://127.0.0.1:7860"

    payload = {
        "sd_model_checkpoint": "ghostmix_v20Bakedvae.safetensors [e3edb8a26f]",
        "prompt": prompt,
        "negative_prompt": negative_prompt,
        "steps":25,
        "seed": f"{seed}",
        "sampler_index": "DPM++ 2M SDE Karras",
        
    }

    response = requests.post(url=f'{url}/sdapi/v1/txt2img', json=payload)

    r = response.json()
    data = json.loads(r["info"])
    seed_value = data["seed"]
    seed = seed_value

    image_path = f"images/image_{index}.png"

    for i in r['images']:
        image = Image.open(io.BytesIO(base64.b64decode(i.split(",",1)[0])))

        png_payload = {
            "image": "data:image/png;base64," + i
        }
        response2 = requests.post(url=f'{url}/sdapi/v1/png-info', json=png_payload)

        pnginfo = PngImagePlugin.PngInfo()
        pnginfo.add_text("parameters", response2.json().get("info"))
        image.save(image_path, pnginfo=pnginfo)
        add_text_to_image(image_path, f'{word_lists[index]}')

Curious boy gazes at the moon.

Oliver building rocket, reaching moon.

Curious boy builds magnificent rocket.

Curious boy gazes at stars.

Excited boy exploring lunar landscape.

Oliver and Luna explore the moon, dancing with joy.

Curious boy building moon rocket.

### Video Compilation

In [15]:
import os
from moviepy.editor import ImageClip, concatenate_videoclips
from moviepy.audio.io.AudioFileClip import AudioFileClip
import os
from moviepy.editor import ImageSequenceClip, concatenate
from moviepy.editor import TextClip
from moviepy.editor import CompositeVideoClip

In [16]:
def create_video(image_dir, audio_file, subtitle_list, output_file):
    # Load the sequence of images
    image_files = sorted(os.listdir(image_dir))

    # Calculate the duration for each image based on the audio duration
    audio = AudioFileClip(audio_file)
    duration_per_image = audio.duration / len(image_files)

    clips = []
    for i, img in enumerate(image_files):
        # Create an ImageClip for each image with the specified duration
        image_path = os.path.join(image_dir, img)
        image = ImageClip(image_path, duration=duration_per_image)

        clips.append(image)

    # Concatenate the clips to create the final video
    final_clip = concatenate_videoclips(clips)

    # Set the audio for the final clip
    final_clip = final_clip.set_audio(audio)

    # Set the frame rate for the final clip
    final_clip.fps = 24

    # Write the video file
    final_clip.write_videofile(output_file, codec='libx264', audio_codec='aac', fps=24)

# Provide the image directory, audio file, and output file path
# only give the
project_dir = '.'
image_directory = f'{project_dir}/images'
audio_file = f'{project_dir}/test.mp3'
output_file = f'{project_dir}/output_video_1.mp4'
subtitle = word_lists


# Call the create_video function
create_video(image_directory, audio_file, subtitle, output_file)

Moviepy - Building video ./output_video_1.mp4.
MoviePy - Writing audio in output_video_1TEMP_MPY_wvf_snd.mp4


MoviePy - Done.
Moviepy - Writing video ./output_video_1.mp4



Moviepy - Done !
Moviepy - video ready ./output_video_1.mp4
